# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [43]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/AI_ML/NMT Internal Lab/Fake News Challenge"

### Loading the Glove Embeddings

In [0]:

with ZipFile(project_path+'/glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd
train_bodies= pd.read_csv(project_path+'/train_bodies.csv')
train_stances= pd.read_csv(project_path+'/train_stances.csv')

In [0]:
dataset=train_bodies.merge(train_stances,on='Body ID')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [242]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


In [243]:
dataset['Stance'].unique()

array(['unrelated', 'agree', 'discuss', 'disagree'], dtype=object)

## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [245]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
from keras.preprocessing.text import Tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:

tokenizer=Tokenizer(num_words=MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [248]:
text_list= list(dataset['articleBody'])
text_list.append(list(dataset['Headline']))
text_list[0]


'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [0]:
tokenizer.fit_on_texts(text_list)

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = list(dataset['articleBody'])
##articles = [] 

In [251]:
texts[1]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [252]:
articles = [] 
[texts[0].split('.')]

[["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday",
  " Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports",
  ' \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth',
  '" House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports',
  ' \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened',
  '\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee',
  ' He said it is still not clear if the meteorite disinte

In [0]:
for text in texts:
  articles.append(text.split('.'))

In [0]:
import re
BAD_SYMBOLS_RE=re.compile('\n')
for j in range(0,len(articles)):
  for i in range(0,len(articles[j])):
    articles[j][i] = re.sub(BAD_SYMBOLS_RE,'',articles[j][i])

In [255]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday",
 " Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports",
 ' Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth',
 '" House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports',
 ' Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee',
 ' He said it is still not clear if the meteorite disintegrated or was bur

In [256]:
from nltk.tokenize import sent_tokenize, word_tokenize

data = "All work and no play makes jack a dull boy, all work and no play"
print(word_tokenize(data))

['All', 'work', 'and', 'no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', ',', 'all', 'work', 'and', 'no', 'play']


## Check 2:

first element of texts and articles should be as given below. 

In [257]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [258]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday",
 " Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports",
 ' Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth',
 '" House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports',
 ' Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee',
 ' He said it is still not clear if the meteorite disintegrated or was bur

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence

In [0]:
import numpy as np
data=np.zeros((len(articles),MAX_SENTS,MAX_SENT_LENGTH))

In [0]:
word_index=tokenizer.word_index

In [262]:
word_index['small']

500

In [263]:
for j in range(0,len(articles)):
  for i in range(0,len(articles[j])):
    if(i<=20):
      words=text_to_word_sequence(articles[j][i])
    for k in range(0,20):
      if(k<=len(words)-1):
        print(words[k])
        data[j,i,k]=word_index[words[k]]
      else:
        data[j,i,k]=0

a
small
meteorite
crashed
into
a
wooded
area
in
nicaragua's
capital
of
managua
overnight
the
government
said
sunday
residents
reported
hearing
a
mysterious
boom
that
left
a
16
foot
deep
crater
near
the
city's
airport
the
associated
press
government
spokeswoman
rosario
murillo
said
a
committee
formed
by
the
government
to
study
the
event
determined
it
was
a
relatively
house
sized
asteroid
2014
rc
which
measured
60
feet
in
diameter
skimmed
the
earth
this
weekend
abc
news
reports
murillo
said
nicaragua
will
ask
international
experts
to
help
local
scientists
in
understanding
what
happened
the
crater
left
by
the
meteorite
had
a
radius
of
39
feet
and
a
depth
of
16
feet
said
humberto
he
said
it
is
still
not
clear
if
the
meteorite
disintegrated
or
was
buried
humberto
garcia
of
the
astronomy
center
at
the
national
autonomous
university
of
nicaragua
said
the
meteorite
could
be
related
to
we
have
to
study
it
more
because
it
could
be
ice
or
rock
he
said
wilfried
strauch
an
adviser
to
the
institute


KeyError: ignored

In [271]:
data[2, :, :]

array([[3.0000e+00, 5.0000e+02, 5.3200e+02, 7.1170e+03, 7.9000e+01,
        3.0000e+00, 3.6640e+03, 3.1800e+02, 5.0000e+00, 4.5760e+03,
        3.6900e+02, 4.0000e+00, 1.6680e+03, 2.8860e+03, 1.0000e+00,
        8.8000e+01, 1.2000e+01, 4.4400e+02, 0.0000e+00, 0.0000e+00],
       [7.2700e+02, 9.4000e+01, 1.0320e+03, 3.0000e+00, 2.9630e+03,
        1.7780e+03, 7.0000e+00, 1.7100e+02, 3.0000e+00, 1.2230e+03,
        1.0950e+03, 2.0620e+03, 7.7900e+02, 1.5400e+02, 1.0000e+00,
        2.9590e+03, 4.7100e+02, 1.0000e+00, 5.3300e+02, 2.2700e+02],
       [8.8000e+01, 1.0330e+03, 4.0270e+03, 2.2750e+03, 1.2000e+01,
        3.0000e+00, 1.0520e+03, 3.2200e+03, 1.9000e+01, 1.0000e+00,
        8.8000e+01, 2.0000e+00, 1.7250e+03, 1.0000e+00, 5.1400e+02,
        1.9560e+03, 1.5000e+01, 9.0000e+00, 3.0000e+00, 3.0350e+03],
       [1.7400e+02, 3.7730e+03, 9.8000e+02, 1.9100e+02, 2.4870e+03,
        4.2000e+01, 6.6890e+03, 1.6650e+03, 1.2100e+03, 5.0000e+00,
        1.3222e+04, 1.8150e+04, 1.0000e+00, 7

### Check 3:

Accessing first element in data should give something like given below.

In [0]:
data[0, :, :]

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
text_headings = list(dataset['Headline'])
articles_headings = [] 

In [0]:
articles = [] 
[text_headings[0].split('.')]

In [0]:
for text in text_headings:
  articles_headings.append(text.split('.'))

In [0]:
import re
BAD_SYMBOLS_RE=re.compile('[^a-zA-Z0-9 ]')
for j in range(0,len(articles_headings)):
  for i in range(0,len(articles_headings[j])):
    articles_headings[j][i] = re.sub(BAD_SYMBOLS_RE,'',articles_headings[j][i])

In [0]:
text_headings[0]

In [0]:
articles_headings[1]

In [0]:
data_headings=np.zeros((len(articles_headings),MAX_SENTS,MAX_SENT_LENGTH))

In [0]:
for j in range(0,len(articles_headings)):
  for i in range(0,len(articles_headings[j])):
    if(i<=20):
      words=text_to_word_sequence(articles_headings[j][i])
    for k in range(0,20):
      if(k<=len(words)-1):
        print(words[k])
        data_heading[j,i,k]=word_index[words[k]]
      else:
        data_heading[j,i,k]=0

In [0]:
data_headings[0,:,:]

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = pd.get_dummies(dataset['Stance'])

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [272]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [273]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)
indices

array([33551, 10954, 43597, ..., 19227,  1957, 47813])

In [0]:
## shuffle the data
data = data[indices]

## shuffle the labels according to data
labels = labels[indices]

In [0]:
data_headings = data_headings[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val,y_train,y_val = train_test_split(data, labels, test_size = 0.2, random_state = 0)

In [0]:
x_heading_train, x_heading_val,y_train,y_val = train_test_split(data_headings, labels, test_size = 0.2, random_state = 0)

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [224]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 4)
(9995, 20, 20)
(9995, 4)


In [0]:
x_train=np.reshape(x_train, (39977, 400))

In [278]:
np.reshape(x_val, (9995, 400))

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [280]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((len(word_index) + 1, 100))


for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


In [237]:
len(word_index) + 1

29071

# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:

from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam


### Model

In [286]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=len(word_index) + 1,
        output_dim=100,
        weights=None,
        trainable=True))

# Recurrent layer
model.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 100)         2907100   
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_9 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 260       
Total params: 2,953,760
Trainable params: 2,953,760
Non-trainable params: 0
_________________________________________________________________


### Compile and fit the model

In [287]:
h = model.fit(x_train, y_train, epochs = 10, batch_size = 50, 
          verbose = 1)## Train the model

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
39977/39977 [==============================] - 523s 13ms/step - loss: 0.3633 - acc: 0.8629
Epoch 2/10
39977/39977 [==============================] - 520s 13ms/step - loss: 0.3547 - acc: 0.8665
Epoch 3/10
39977/39977 [==============================] - 519s 13ms/step - loss: 0.3526 - acc: 0.8666
Epoch 4/10
39977/39977 [==============================] - 518s 13ms/step - loss: 0.3514 - acc: 0.8667
Epoch 5/10
  100/39977 [..............................] - ETA: 8:40 - loss: 0.3601 - acc: 0.8650

KeyboardInterrupt: ignored

## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)